<font size="15"><b>Project 乐吧泡菜味薯片 *!!!
    
<img src="https://world-fr.openfoodfacts.org/images/products/997/022/950/1521/front_zh.3.full.jpg" 
     alt="乐吧泡菜味薯片 - Produit - zh"
     width=200
     />
    
*translates to Fatmerican fries ?

# import libs

In [ ]:
#pip install missingno wordcloud

<font color='darkblue' size=16>MITO : Python data analysis in seconds

https://www.trymito.io/
https://docs.trymito.io/getting-started/installing-mito

Mito allows the user to code much faster, by giving them a spreadsheet interface for Python that generates code.

In [1]:
#python -m pip install mitoinstaller
#python -m mitoinstaller install

import mitosheet
mitosheet.sheet()


MitoWidget(analysis_data_json='{"analysisName": "id-neczppaakb", "analysisToReplay": null, "code": [], "stepSu…

In [2]:
# create some simple data to display
train_stations = pd.DataFrame({'Zip': [21001, 97321, 49224, 87102, 24910, 22301], 'City': ['Aberdeen', 'Albany', 'Albion', 'Albuquerque', 'Alderson', 'Alexandria'], 'State': ['MD', 'OR', 'MI', 'NM', 'WV', 'VA'], 'Ticket_Office': ['N', 'Y', 'N', 'Y', 'N', 'Y']})
demographics = pd.DataFrame({'Zip': [21001, 97321, 49224, 87102, 24910, 22301], 'Median_Income': [53979.0, 112924.0, 37556.0, 28388.0, 30914.0, 54087.0], 'Mean_Income': [66169.0, 147076.0, 50371.0, 39529.0, 40028.0, 64068.0], 'Pop': [18974.0, 11162.0, 14900.0, 22204.0, 5383.0, 19504.0]})

# render the Mitosheet with the data
mitosheet.sheet(train_stations, demographics)

MitoWidget(analysis_data_json='{"analysisName": "id-urbqrkasfb", "analysisToReplay": null, "code": [], "stepSu…

In [11]:
import pandas as pd
#from pandasgui import show


import missingno as msno

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import seaborn as sns

import numpy as np
from scipy.stats import kurtosis,skew 
from sklearn.cluster import KMeans

from wordcloud import WordCloud # Generate word cloud
import imageio


import os,glob,datetime

from ipywidgets import widgets 
from ipywidgets.widgets import HTML,Output,HBox,Layout
from ipywidgets import AppLayout,Button,Label


<IPython.core.display.Javascript object>

# Custom functions

## Function display Dates Col

In [ ]:
def displayDate(df0,col) :
    left = Output()
    right = Output()
    hbox = HBox([left,right])

    with left :
        describeNumCol(df0,col)
    with right :
        fig,[ax,ax1] = plt.subplots(2,1)

        ax1.sharex = ax

        sns.boxplot(x=df0[col].view(np.int64) ,
                    notch=True,
                    color=(0.1,0,0.2,0.5),
                    ax = ax
                    ).set_title(f"distribution of {col}")


        ax.xaxis.set_visible(False)

        # axe 1 ---------------------------------------------
        df0[col].view(np.int64).plot.hist(ax=ax1)

        ax1.set_ylabel("Nb observations")

        #ax1.xaxis.set_ticks(df0[col]) # should resolve warning message FixedFormatter should only be used together with FixedLocator but does not
        ax1.xaxis.set_major_locator(ticker.AutoLocator())
        ax1.set_xticklabels(labels=df0[col], rotation=45, ha='right')

        plt.show()

    display(hbox)


## Function describeCatCol

In [ ]:
def cutoffLF(df_values_desc,level=0.01) :

    #let's cut off low frequencies < 1% and keeping top 10 only
    focus_desc1 = df_values_desc[df_values_desc.freq > level ].sort_values("freq")
    lost1 =df_values_desc[df_values_desc.freq < level ]
    
    display((HTML(f"""<font color="red"></b>Cutting off low frequency values(<{level})</font></b> 
    grouping {len(lost1)} values in Others category
    ignored values are shown in word cloud below: """)))
    
    # other option : keeping top 10 + others / 
    # maybe it would be better to look for a good trade off between top n and quantile 
    #=> au filal on fait du PCA artisanal :(
    
    focus_desc = focus_desc1.head(10)
    lost2 = focus_desc1.iloc[10: , :]
    lost = pd.concat([lost1,lost2])
    
    focus_desc.loc["other"] = [lost["nb_onservations"].sum(),#nb_observations
              lost["nb_onservations"].sum()/focus_desc.nb_onservations.sum(),#freq
              ]
    
    focus_desc["cumsum"] = (focus_desc.nb_onservations/focus_desc.nb_onservations.sum()).cumsum()


    lost_asString = ' '.join(lost.index.to_list() )
    if len(lost_asString) >0 :
        wc = WordCloud(width = 800, height = 300,).generate(lost_asString)

        # plot the WordCloud image                        
        plt.axis('off')
        plt.imshow(wc)
        plt.show() 
    else : 
        display(HTML(f"no value has been removed"))

    return focus_desc

In [ ]:
def describeCatCol(df,colName,cutoff=0.01,log=False,l=12,L=6) :
    """
    plot the distribution of values for a given variable
    args : 
    - cutoff : groupe all frequencies < cutoff in teh category Others (default = 1%)
    - log : display y axes in log 
    - l : figure height
    - L : figre width
    """

    n = len(df[colName])
    
    
    df_values_desc = pd.DataFrame(
                        [df[colName].value_counts(),
                         df[colName].value_counts()/n,
                        ]
                        ).transpose()
    df_values_desc.columns=["nb_onservations","freq"]

    
    display(HTML(f"There are {n} unique values, {100*df[colName].isna().sum()/n:.2f}% missing values"))

    focus_desc = cutoffLF(df_values_desc,cutoff)

    #sempre fare la bella figura
    fig,ax = plt.subplots(figsize=(l,L) )
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    plt.title(f"Description of {colName}",
            fontdict={
                'fontsize': 30,
                'fontweight' : "bold" ,
                'verticalalignment': 'baseline',
                'horizontalalignment': 'center'},
            pad =50,
            )

    # grid specification
    gs = fig.add_gridspec(2,1)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[1, 0])

    # ax1 : bar plot of cumsum
    ax1.set_title(f'Value distrubution for {colName} (frequency cumsum)', fontsize=12)
    ax1.set_xlabel(f'{colName} possible values', fontsize=20)
    ax1.set_xticklabels( 
                    labels = focus_desc.index.str.slice(0,20),
                    rotation=45,
                    fontsize=14
                    )
    ax1.set_ylabel('frequency cumsum', fontsize=20) 
    if log :
        ax1.set_yscale('log')


    ax1.bar(x=focus_desc.index,
            height=focus_desc["cumsum"]
           )
    ax1.plot(focus_desc["cumsum"],
             color='red'
            )
    #show values on top of bars 
    for bars in ax1.containers:
        ax1.bar_label(bars)

    # ax2 : bar plot of freq 
    sns.barplot(x="freq", 
              y=focus_desc.index, 
              data=focus_desc,
              palette="flare",
              orient='h',
              ax=ax2
              )
    ax2.set_xlabel('Frequency', fontsize=20)
    ax2.set_ylabel(f'{colName} possible values', fontsize=20) 


    plt.subplots_adjust(left=None, bottom=None, right=2, top=None, wspace=None, hspace=1)
    #left  = 0.125  # the left side of the subplots of the figure
    #right = 0.9    # the right side of the subplots of the figure
    #bottom = 0.1   # the bottom of the subplots of the figure
    #top = 0.9      # the top of the subplots of the figure
    #wspace = 0.2   # the amount of width reserved for blank space between subplots
    #hspace = 0.2   # the amount of height reserved for white space between subplots

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

    return df_values_desc

## Function describeNumCol

In [ ]:
def describeNumCol (df,colName) :
    """
    describe a numeric column with basic stats and graphics
    """

    n = len(df[colName])

    # calculate sample kurtosis ###################################################
    
    gamma1 = kurtosis(df[colName].values.astype(np.int64), bias=False)
    #The kurtosis of a normal distribution is 3.
    #kurtosis less than 3 : playkurtic, which means it tends to produce fewer and less extreme outliers than the normal distribution.
    #kurtosis greater than 3, : leptokurtic, which means it tends to produce more outliers than the normal distribution.
    if gamma1 <2.9 : 
        interp_gamma1 =  "playkurtic (concentrated)"
    elif gamma1 >3.1 :
        interp_gamma1 =  "leptokurtic (many outliers)"
    else : 
        interp_gamma1 = "distribution is normal"
    ################################################################################

    # calculate sample skewness  ###################################################
    
    gamma2 = skew(df[colName].values.astype(np.int64), bias=False)
    #A negative skew = the tail is on the left side of the distribution, which extends towards more negative values.
    #A positive skew = the tail is on the right side of the distribution, which extends towards more positive values.
    #A value of zero = there is no skewness in the distribution at all, meaning the distribution is perfectly symmetrical.
    if gamma2 <-0.2 : 
        interp_gamma2 =  "tail is on the left side of the distribution"
    elif gamma2 >0.2 :
        interp_gamma2 =  "tail is on the right side of the distribution"
    else : 
        interp_gamma2 = "distribution is symmetrical"
    #################################################################################


    # calculate Lorentz and Gini  ###################################################
    
    lorenz = np.cumsum(np.sort(df[colName].values.astype(np.int64))) / df[colName].values.astype(np.int64).sum()
    lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0
    AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
    S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
    gini = 2*S
    gini
    ##################################################################################
    
    # report all indicators in a dataframe  ##########################################
    full_desc = df[[colName]].describe(datetime_is_numeric=True)
    full_desc.loc["Unique values%"] = f"{100*df[colName].nunique()/n:.2f}"
    full_desc.loc["Missing values%"] = f"{100*df[colName].isna().sum()/len(df[colName]):.2f}"
    full_desc.loc["Kurtosis"] = gamma1
    full_desc.loc["Kurto desc"] = interp_gamma1
    full_desc.loc["Skew"] = gamma2
    full_desc.loc["Skew desc"] = interp_gamma2
    full_desc.loc["Gini"] = gini
    ##################################################################################
    ##################################################################################
    
    display(full_desc)

In [ ]:
def displayNumCol(df,colName) :
    
    dist_series = df[colName].value_counts()
    
    fig,ax = plt.subplots(figsize=(20,15) )
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    plt.title(f"Variable {colName}",
            fontdict={
                'fontsize': 24,
                'fontweight' : "bold" ,
                'verticalalignment': 'baseline',
                'horizontalalignment': 'center'},
            pad =50,
            )

    # grid specification
    gs = fig.add_gridspec(3,2)
    ax1 = fig.add_subplot(gs[0, 0])

    ax2 = fig.add_subplot(gs[1, 0])
    ax2.sharex = ax1

    ax3 = fig.add_subplot(gs[0, 1])
    ax3.sharey = ax1

    ax4 = fig.add_subplot(gs[1, 1])

    ax5 = fig.add_subplot(gs[2, 1])


    # axe 1 ---------------------------------------------
    ax1.bar(dist_series.index,dist_series, color='b')
    ax1.set_title('Distribution of values')
    ax1.set_ylabel("Nb observations")
    ax1.xaxis.set_tick_params(rotation=45)

    # axe 2 ---------------------------------------------
    sns.histplot(x=dist_series.index, kde=True, stat='density',ax=ax2 )
    ax2.xaxis.set_visible(False)
    ax2.set_title('Density KDE')
    ax2.set_ylabel(colName)
    ax2.invert_yaxis()

    # axe 3 ---------------------------------------------
    sns.histplot(y=dist_series, kde=True, stat='density',ax=ax3 )
    ax3.xaxis.set_visible(False)
    ax3.set_title('Density KDE')
    ax3.set_ylabel("Nb Observation")

    # axe 4 ---------------------------------------------
    sns.boxplot(x=df[colName], ax =ax4,notch=True,color=(1,0,1,0.5))
    ax4.set_xlabel(colName)
    ax4.set_title('Distribution of values Boite a mous')


    # axe 5 -----------------------------------------------

    xaxis = np.linspace(0-1/n,1+1/n,len(lorenz)) #one segment of len n for each individual + 1 additional segment at the origine 0. 
    #first segment starts at 0-1/n, las ends at 1+1/n.
    ax5.plot(xaxis,lorenz,drawstyle='steps-post')
    ax5.plot([0,1], [0,1]) #bisector
    ax5.set_title('Lorentz distribution')
    
    plt.show()


In [ ]:
def screenNumCol (df,colName) :
    ## prepare widgets outputs  ***************************************
    graph = widgets.Output()
    table = widgets.Output(layout=widgets.Layout(margin='0 50px 0 0'))
    hLayout = widgets.HBox([table,graph])
    
    # display ---------------------------------------------

    with graph :
        graph.clear_output()
        displayNumCol(df,colName)
    with table :
        table.clear_output()
        describeNumCol(df,colName)

    display(hLayout)

## Method Convert col dtype

In [ ]:
def convert_col(df0,col,target_type,loss_threshold=3,inplace=True) :
    """ converts a column of a given dataframe to a target type after verifying the information loss <3% (default)    
    if compliant with the max information loss condition : changes are made in place  
    otherwise returns the lost information table
    """

    display(HTML(f"""<h2>{col} convertion"""))

    #nb na before 
    nb_na_before = df0[col].isna().sum()
    
    #convert col to num
    df1 = pd.DataFrame()
    
    if target_type in ["float","int","string"] :
        df1[col] = df0[col].astype(target_type)
    
    elif target_type == "datetime" : 
        df1[col] = pd.to_datetime(df0["created_t"] , unit='s',errors = "coerce")
        
    else :
        display("error : unrecog type")
    
    
    #nb na after
    nb_na_after = df1[col].isna().sum()

    ap = pd.concat([df0[col],df1[col],],axis=1)
    ap.columns=[f"before","after"]

    display (HTML(f"""Nb na diff : {nb_na_before-nb_na_after}, 
    before = {nb_na_before}, after nb na = {nb_na_after}
    """))
    display(ap.sample(4))
    display(ap.dtypes)
    
    #report loss information
    lost = df0[df0[col].notna() & df1[col].isna()]
    
    if (nb_na_before-nb_na_after)*100/len(df0) < loss_threshold :
        df0[col] = df1[col]
        display(HTML("<div class='alert alert-block alert-success'> Valide transformation, registered in place"))

    else :
        display(HTML("<div class='alert alert-block alert-danger'> Invalide transformation - too much information lost"))

    if len(lost) > 0 :
        display(HTML("<font color='red'>Information lost (sample):"))
        display(lost.sample(4))
        
    #describeNumCol(df0,col)
    
    return lost


## clean1

In [ ]:
# for future usage check all col fill rate , unique value rate and duplicates
def clean1(df_raw) :
    """
for a given dataframe, on a copy :
      - Drops empty variables
      - Strip categorial variables

Returns the dataframe and 2 descriptions dictionary (basic stats + convertion errors)  
  """

    df = df_raw.copy()

    nb_num = 0
    nb_cat = 0

    nb_dup = df.duplicated().sum()
    dupRate = nb_dup/len(df)

    (nb_rowsBefore,nb_colsBefore) = df.shape

    #print(f"""Sample Data BEFORE : """)
    #display(df.head(3))

    newRow = {}
    desc_dict = {} #keys=["dataType","nbFilled","fillRate%","uniqueVal","uvRate%"])
    errors_dict={}


    display(HTML(f"""
    <h1><strong>  Cleaining df : </h1>

    <h2>Renaming columns (strip and replace white spaces with underscores) </h2></strong>
    """))

    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')

    ###################################################################################

    display(HTML(f"""
    <strong>
    <h2>droping empty columns</h2></strong>
    """))


    for col in df : 

        dataType= df[col].dtype
        nbFilled = df[col].notna().sum()
        fillRate = 100*nbFilled / len(df[col])
        uniqueVal = df[col].nunique()
        uvRate = 100*uniqueVal /  len(df[col])
        nullVal = df[col].isna().sum()
        nullRate = 100*nullVal /  len(df[col])


        if fillRate == 0 :
            display(HTML(f"""<font color='red'>Droping empty column : {col} </font>"""))
            df.drop([col], axis=1,inplace=True)
            #desc[col] = "empty variable dropped"
            break

            
        newRow = {"dataType" : dataType,
                "nbFilled":nbFilled,
                "fillRate%":fillRate,
                "nbNull":nullVal,
                "nullRate%":nullRate,
                "uniqueVal":uniqueVal,
                "uvRate%":uvRate,
                }


        ##################################################
        #check if column could be converted to num
        
        numVersion_series = pd.to_numeric(df[col],errors='coerce')
        fillRateNum = 100*numVersion_series.notna().sum()/len(numVersion_series)
        scoreNum = round(fillRateNum*100/fillRate)

        if scoreNum > 80 :     

            errors_dict[col] = { "scoreNum" : scoreNum ,
                            "lostData_series" : df[col][numVersion_series.isna()] }
            display(HTML(f"""<font color='blue'>
if {col} were converted to numeric => fillrate {fillRateNum:.2f}  vs {fillRate:.2f}
check errors_dict for further treatment.     
"""))
        nb_num += 1

        #check if string
        if pd.api.types.is_string_dtype(df[col] ) :
            
            nb_cat += 1

            try:
                df[col] = df[col].str.strip() #remove white spaces before and after 

            except Exception as e : 
                print(f"""
    Error {e}
    while stripping col : {col} as string           
              """)


            finally :
                print(f"{col} striped")
                

                if "last_modified_t" in col :
                    print(f"""
_____________________________________________
Checking Column : {col}
*****************************
antes : {df_raw[col]}
****************************
despues : {df[col]}
_____________________________________________

                    """)



        else : 
            nb_num += 1

        desc_dict[col] = newRow

    ###################################################################################

    display(HTML(f"""
    <h2>Dropping empty rows </h2></strong>
    """))

    ## After
    df.dropna(how='all',inplace=True)
    (nb_rowsAfter,nb_colsAfter) = df.shape

    ###################################################################################

    display(HTML(f"""
    <h1>Sample Data AFTER Cleaning :</h1> 
    <h3>
    <ul>
    <li>number full duplicated rows : {nb_dup} - {dupRate:.2f}% ,{nb_rowsAfter} ({100*nb_rowsAfter/nb_rowsBefore:.0f}%) were not empty </li> 
    <li>{nb_rowsAfter} individuals - dropped <font color='red'>{100*(nb_rowsBefore-nb_rowsAfter)/nb_rowsBefore:.0f}% </font></li>
    <li>{nb_colsAfter} variables - dropped <font color='red'>{100*(nb_colsBefore-nb_colsAfter)/nb_colsBefore:.0f}%</font>
        <ul>
             <li>{nb_num} numeric </li>
             <li> {nb_cat} categorial</li> 
        </ul>
    </li>
    </h3> 
    """))
    display(df.head(3))

    msno.matrix(df)
    plt.title("MISSING VALUES",fontdict={'color':  'darkblue',
                                     'weight': 'normal',
                                     'size': 32,
                                     })
    plt.show()

    return df,desc_dict,errors_dict

## screenDF (df description)

In [ ]:
def screenDF(df1_desc,L=25,l=25) :
    
    """displays metada data fillrate unique values and nullrate 
    """

    fig,(ax0,ax1,ax2) = plt.subplots(1,3,figsize=(L,l))

    ax1.sharey=ax0
    ax1.yaxis.set_visible(False)
    ax2.sharey=ax0
    ax2.yaxis.set_visible(False)

    sns.barplot(data =df1_desc,x='fillRate%',y=df1_desc.index,ax=ax0)
    sns.barplot(data =df1_desc,x='uvRate%',y=df1_desc.index,ax=ax1)
    sns.barplot(data =df1_desc,x='nullRate%',y=df1_desc.index,ax=ax1)

    ax0.axvline(x=10,color='red')
    ax2.axvline(x=10,color='red')

    ax0.set_title("fill rate %")
    ax0.yaxis.set_label("variables")
    ax1.set_title("unuqiue val rate %")
    ax2.set_title("null rate %")

    plt.plot()

# Load data and first cleaning

## Load

In [ ]:
%%script bash

function getdata () {
    wget https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip
    mkdir data
    unzip -d data fr.openfoodfacts.org.products.csv.zip
    ls data 
    echo ------------------------------------
    wc -l data/fr.openfoodfacts.org.products.csv
    head data/fr.openfoodfacts.org.products.csv
    tail data/fr.openfoodfacts.org.products.csv
    
}


[[ -f data/fr.openfoodfacts.org.products.csv ]] && echo data already downloaded || getdata

In [12]:
filename = "data/fr.openfoodfacts.org.products.csv"

df0_raw = pd.read_csv(filename,sep='\t')
display(HTML("Data loaded in dataframe"))
df0_raw.sample(3)

/home/zak/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


HTML(value='Data loaded in dataframe')

code                                                url  \
44204     41318020304  http://world-fr.openfoodfacts.org/produit/0041...   
228458  3364699325613  http://world-fr.openfoodfacts.org/produit/3364...   
114160  0257776008949  http://world-fr.openfoodfacts.org/produit/0257...   

                creator   created_t      created_datetime last_modified_t  \
44204   usda-ndb-import  1489060907  2017-03-09T12:01:47Z      1489060907   
228458         teolemon  1388849679  2014-01-04T15:34:39Z      1455394450   
114160          tacinte  1472907037  2016-09-03T12:50:37Z      1472907093   

       last_modified_datetime             product_name generic_name quantity  \
44204    2017-03-09T12:01:47Z  Classic Cut Green Beans          NaN      NaN   
228458   2016-02-13T20:14:10Z          Concombre seche          NaN    100 g   
114160   2016-09-03T12:51:33Z                      NaN          NaN      NaN   

        ... ph_100g fruits-vegetables-nuts_100g  \
44204   ...     NaN                         NaN   
228458  ...     NaN                         NaN   
114160  ...     NaN                         NaN   

       collagen-meat-protein-ratio_100g cocoa_100g chlorophyl_100g  \
44204                               NaN        NaN             NaN   
228458                              NaN        NaN             NaN   
114160                              NaN        NaN             NaN   

       carbon-footprint_100g nutrition-score-fr_100g nutrition-score-uk_100g  \
44204                    NaN                     NaN                     NaN   
228458                   NaN                     NaN                     NaN   
114160                   NaN                     NaN                     NaN   

       glycemic-index_100g water-hardness_100g  
44204                  NaN                 NaN  
228458                 NaN                 NaN  
114160                 NaN                 NaN  

[3 rows x 162 columns]

In [ ]:
df0= df0_raw.copy()

In [ ]:
show(df0)

## Infering dtypes :
Auto (pandas) did not provide any suggestion.

Man dtypes infer of non numerical variables rises a number of functional issues :
- how to handle foreign policies
- are controvertial components traced and even tracable

### Auto with infer_objects()
No change suggested

In [ ]:
before = df0.dtypes
before.name="dtype_before"

df1 = df0.infer_objects()

after = df1.dtypes
after.name="dtype_after"

df2 = pd.merge(before,after,right_index=True, left_index=True)

display(HTML("""<h2>Auto infer types report : """,
             layout=Layout(object_position='center',
                           border='solid 1px black',
                          )
            )
       )
display(HTML("""<h3>Changes : """))
display(df2[df2.dtype_after != df2.dtype_before])

display(HTML(value='<h2>Before and after dtype auto infer',))
left_output = Output()
centerRight_output = Output()
centerLeft_output = Output()
right_output = Output()

with left_output :
    before.value_counts().plot.pie()
    plt.show()
    
with centerLeft_output :
    display(HTML("<== Before "))
    display(before.value_counts())
    
with centerRight_output :  
    display(HTML('<div text-align: right >After  ==> '))
    display(after.value_counts())
    
with right_output :
    after.value_counts().plot.pie()
    plt.show()

hbox = HBox([left_output,centerLeft_output,centerRight_output,right_output],
          height="100%", 
          width="100%")
display(hbox)

In [ ]:
del df1,df2,after

### Manual with select_dtypes


<div class ="alert alert-block alert-danger">
"villecomtal-sur-arros-gers-france" and "saint-just-chaleyssin-isere-france" could be unreliable sources at the expence of some dairy products.
</div>



#### first round : non numerical variables

Convertion scheme 1 :

- <font color="green"><b>To numeric :
    - code would be a numerical id 
    
- <font color="blue"><b>To string :
    - product_name and url should be strings and could be coupled with code for stronger id provided they are consistantly observed
    - main_category	 and main_category_fr
    - additives_tags,additives_fr, ingredients_from_palm_oil_tags, ingredients_that_may_be_from_palm_oil_tags	
    - nutrition_grade_fr is the nutriscore

- <font color="orange"><b>To datetime :
    - created_datetime, last_modified_datetime are formated like %y/%m/%dT%H:%M:%SZ (2017-03-09T20:49:44Z)
    - from timestamps : created_t, last_modified_t, 
    
- <font color="red"><b>To drop :
    - a priori not relevant : image_url, image_small_url, states_fr, states_tags
    - a posteriori not relevant : "created_t", "last_modified_t","created_datetime","last_modified_datetime"

In [ ]:
df_notNum = df0.select_dtypes(exclude='float')
display(HTML(f"""<h2>Number of non numerical values : {df_notNum.shape[1]}
<h3>Samples :"""))
display(HTML(
    df_notNum.sample(3).transpose().to_html()
))

In [ ]:
lost = pd.DataFrame()

##### convertions to num

In [ ]:
col = "code" 
target_type = "float"

lost1 = convert_col(df0,col,target_type)
lost = lost.append(lost1)

In [ ]:
describeNumCol(df0,col) # not relevant for ids

##### to datetime

In [ ]:
target_type = "datetime"

for col in ["created_t", "last_modified_t","created_datetime","last_modified_datetime"] :
    lostn = convert_col(df0,col,target_type) 
    lost = lost.append(lostn)
    

In [ ]:
for col in ["created_t", "last_modified_t","created_datetime","last_modified_datetime"] :
    displayDate(df0,col)

*About matplotlib locator : https://matplotlib.org/3.1.1/gallery/ticks_and_spines/tick-locators.html*

<div class="alert alert-block alert-danger"> Considering timestamps : all data must have been uploaded in bulk around march 09 2017.
those variables will not be relevant for further analysis and will be dropped.
</div>

###### About lost info

In [ ]:
lost.drop_duplicates(keep="last",inplace=True)
#unreliable sources :
lost.creator.value_counts()

In [ ]:
#product out of scope 
outofscope = df0.iloc[lost.creator.index]
out = outofscope.generic_name.str.strip().value_counts()
out.index = out.index.str[:30]
out

#### second round : remaining non numerical variables  

In [ ]:
df0.drop(['image_url', 'image_small_url', 'states_fr', 'states_tags'], axis=1,inplace=True)
df0.drop(["created_t", "last_modified_t","created_datetime","last_modified_datetime"],axis=1,inplace=True)

In [ ]:
df_notNum = df0.select_dtypes(exclude=['float','datetime64[ns]'])
display(HTML(f"""<h2>Number of non numerical values : {df_notNum.shape[1]}
<h3>Samples :"""))
display(HTML(
    df_notNum.sample(3).transpose().to_html()
))

##### quantity
to be continued

In [ ]:
#test with quantity
df0.quantity.value_counts()

In [ ]:
#remove kg 
test = df0[df0.quantity.str.contains("g").fillna(False)].quantity

#review possible loss of information :
testdf = df0.iloc[test.index][["code","creator","quantity"]]
focusdf = testdf.quantity.str.split("g",expand=True)

focusdf[
    focusdf[2].notna() |
    focusdf[11].notna() 
]

##### allergens

In [2]:
df0["allergens"] = df0["allergens"].fillna("None")

NameError: name 'df0' is not defined

In [ ]:
df0["allergens"].value_counts()

In [ ]:
describeCatCol(df0,"allergens",cutoff=0,l=1,L=6)

##### product_name
weird names : 乐吧泡菜味薯片

<div class="alert alert-block alert-danger">About China and US policies

In [ ]:
df0[df0.product_name == "乐吧泡菜味薯片"].url

##### controversial components
 <a href="https://en.wikipedia.org/wiki/Monosodium_glutamate">MSG

In [ ]:
MSG = ["Monosodium glutamate", "sodium glutamate",
"Sodium 2-aminopentanedioate",
"Glutamic acid", "monosodium salt", "monohydrateL-Glutamic acid", "monosodium salt", "monohydrate"
"L-Monosodium", "glutamate", "monohydrate",
"Monosodium L-glutamate monohydrate","MSG monohydrate","Sodium glutamate monohydrate UNII-W81N5U6R6U"
,"Flavour enhancer" ,"E621"]

In [ ]:
df0.columns.isin(MSG)

## duplicated ids

In [ ]:
ident = "code"

### methode A : value_count
found 134 duplicated rows

In [ ]:
# duplicates
dup = df0[ident].value_counts()
dup.name="nbdup"
dup = dup[dup > 1]

duplicatedRows = df0[df0.code.isin(dup.index.tolist())].sort_values("code")

naRows = df0[df0.code.isna()]


display(HTML(f""" Considering variable {ident} : {len(dup.index.tolist())+len(naRows)} duplicates
found {len(dup.index.tolist())} duplicated id and {len(naRows)} nan 
"""))

display(HTML("Sample na Rows :"))
if len(naRows) >0 :
    display(naRows.sample(5))
else : display("**************")

display(HTML("Sample duplicated Rows :"))
if  len(dup.index.tolist()) >0 :
    display(duplicatedRows.sample(5))
else : display("**************")


### methode b : duplicated
found 133

In [ ]:
dupB = df0[df0["code"].duplicated()].sort_values("code")

display(HTML(f"""found {len(dupB)} duplicated ids"""))

duplicatedRows = df0[df0.code.isin(dupB.code.tolist())].sort_values("code")
display(duplicatedRows)

### deduplicated data :
remove nan ids as no possible data recovery
keep only one version of the duplicates 
<div class="alert alert-block alert-danger"> 
(to do : choose the best options considering either :
    
    - fill rate or 
    - values consistancy or
    - last value considering the timestamps

In [ ]:
#remove na 
df0.dropna(subset=["code"],inplace=True)
df0.drop_duplicates(subset=["code"],keep="last",inplace=True)


### Function describeNumCol

In [ ]:
def describeNumCol (df,colName) :
  """
  describe a numeric column with basic stats and graphics
  args : df,colName 
  """

  dist_series = df[colName].value_counts()
  n = len(df[colName])

  #calculate sample kurtosis
  gamma1 = kurtosis(df[colName], bias=False)
  #The kurtosis of a normal distribution is 3.
  #kurtosis less than 3 : playkurtic, which means it tends to produce fewer and less extreme outliers than the normal distribution.
  #kurtosis greater than 3, : leptokurtic, which means it tends to produce more outliers than the normal distribution.
  if gamma1 <2.9 : 
    interp_gamma1 =  "playkurtic (concentrated)"
  elif gamma1 >3.1 :
    interp_gamma1 =  "leptokurtic (many outliers)"
  else : 
    interp_gamma1 = "distribution is normal"
  
  #calculate sample skewness
  gamma2 = skew(df[colName], bias=False)
  #A negative skew = the tail is on the left side of the distribution, which extends towards more negative values.
  #A positive skew = the tail is on the right side of the distribution, which extends towards more positive values.
  #A value of zero = there is no skewness in the distribution at all, meaning the distribution is perfectly symmetrical.
  if gamma2 <-0.2 : 
    interp_gamma2 =  "tail is on the left side of the distribution"
  elif gamma2 >0.2 :
    interp_gamma2 =  "tail is on the right side of the distribution"
  else : 
    interp_gamma2 = "distribution is symmetrical"


  #calculate Lorentz and Gini  
  lorenz = np.cumsum(np.sort(df[colName])) / df[colName].sum()
  lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0
  AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
  S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
  gini = 2*S
  gini


  #prepare outputs
  graph = widgets.Output()
  table = widgets.Output(layout=widgets.Layout(margin='0 50px 0 0'))
  hLayout = widgets.HBox([table,graph])

  #sempre fare la bella figura
  fig,ax = plt.subplots(figsize=(20,15) )
  ax.xaxis.set_visible(False)
  ax.yaxis.set_visible(False)
  plt.title(f"Variable {colName}",
            fontdict={
                'fontsize': 24,
                'fontweight' : "bold" ,
                'verticalalignment': 'baseline',
                'horizontalalignment': 'center'},
            pad =50,
            )
  
  # grid specification
  gs = fig.add_gridspec(3,2)
  ax1 = fig.add_subplot(gs[0, 0])

  ax2 = fig.add_subplot(gs[1, 0])
  ax2.sharex = ax1

  ax3 = fig.add_subplot(gs[0, 1])
  ax3.sharey = ax1

  ax4 = fig.add_subplot(gs[1, 1])

  ax5 = fig.add_subplot(gs[2, 1])


  # axe 1 ---------------------------------------------
  ax1.bar(dist_series.index,dist_series, color='b')
  ax1.set_title('Distribution of values')
  ax1.set_ylabel("Nb observations")
  ax1.xaxis.set_tick_params(rotation=45)

  # axe 2 ---------------------------------------------
  sns.histplot(x=dist_series.index, kde=True, stat='density',ax=ax2 )
  ax2.xaxis.set_visible(False)
  ax2.set_title('Density KDE')
  ax2.set_ylabel(colName)
  ax2.invert_yaxis()

  # axe 3 ---------------------------------------------
  sns.histplot(y=dist_series, kde=True, stat='density',ax=ax3 )
  ax3.xaxis.set_visible(False)
  ax3.set_title('Density KDE')
  ax3.set_ylabel("Nb Observation")

  # axe 4 ---------------------------------------------
  sns.boxplot(x=df[colName], ax =ax4,notch=True,color=(1,0,1,0.5))
  ax4.set_xlabel(colName)
  ax4.set_title('Distribution of values Boite a mous')
 

  # axe 5 -----------------------------------------------

  xaxis = np.linspace(0-1/n,1+1/n,len(lorenz)) #one segment of len n for each individual + 1 additional segment at the origine 0. 
  #first segment starts at 0-1/n, las ends at 1+1/n.
  ax5.plot(xaxis,lorenz,drawstyle='steps-post')
  ax5.plot([0,1], [0,1]) #bisector
  ax5.set_title('Lorentz distribution')



  # display ---------------------------------------------

  with graph :
    graph.clear_output()
    plt.show()
  with table :
    table.clear_output()
    full_desc = df[[colName]].describe()
    full_desc.loc["Unique values"] = n 
    full_desc.loc["Missing values%"] = f"{100*df[colName].isna().sum()/len(df[colName]):.2f}"
    full_desc.loc["Kurtosis"] = gamma1
    full_desc.loc["Kurto desc"] = interp_gamma1
    full_desc.loc["Skew"] = gamma2
    full_desc.loc["Skew desc"] = interp_gamma2
    full_desc.loc["Gini"] = gini

    display(full_desc)


  display(hLayout)

# Cleaning DF0 => DF1 => DF2

In [ ]:
df1["last_modified_t"].isna().sum(),df0["last_modified_t"].isna().sum()

In [ ]:
pd.api.types.is_numeric_dtype(df0["last_modified_t"])

In [ ]:
errors_dict

In [ ]:
df1,desc_dict,errors_dict = clean1(df0)

In [ ]:
df1_desc = pd.DataFrame(desc_dict).transpose().sort_values(by="fillRate%",ascending=False)
df1_desc

screenDF(df1_desc)

In [ ]:
df1_desc[["fillRate%","nullRate%","uvRate%"]]

## timestamps : last_modified_t

In [ ]:
#timestamps and datetimes 
df0["last_modified_t_isdig"] =  pd.to_numeric( df0["last_modified_t"],errors='coerce'   )

#lost data if timestamp to numeric
imparedData = df0 [df0["last_modified_t_isdig"].isna() ].dropna(axis=1)
display(imparedData.sample(5) )
display(imparedData.describe() )


In [ ]:
l = ' | '.join(imparedData.columns.tolist())
f"{len(imparedData.columns)} entries : {l}"

In [ ]:
df = imparedData

colToScan = ["purchase_places","last_modified_t","product_name","creator","cities","cities_tags","stores"	] #4

g = len(colToScan)

nblig = int(g/6)+1
nbcol = 6

fig,ax = plt.subplots(figsize=(20,15))
spec = fig.add_gridspec(ncols=nblig, nrows=nbcol)

i=0

wordcloud = WordCloud(background_color='white', width = 300, height=300, margin=2)


for i in range(len(colToScan)) : 

    axi = fig.add_subplot(spec[i])
    val_series = df[colToScan[i]].value_counts()

    axi.set_title(colToScan[i])
    text = ' '.join(imparedData.columns.tolist())

    #  wordcloud.generate(' '.join(imparedData[colToScan[i]].tolist()))
    #  ax[i].imshow(wordcloud)

    try :
        wordcloud.fit_words(val_series.to_dict())
        axi.imshow(wordcloud)

        axi.axis("off")
    except Exception as e :
        
        print(f"""_________________________________________________
handling ax {i} - {colToScan[i]}
******************************************
{val_series}""")

  

plt.axis("off")
plt.axis("off")
plt.show()

In [ ]:
def annotate_axes(ax, text, fontsize=18):
    ax.text(0.5, 0.5, text, transform=ax.transAxes,
            ha="center", va="center", fontsize=fontsize, color="darkgrey")
    
fig = plt.figure(figsize=(5.5, 3.5), constrained_layout=True)

spec = fig.add_gridspec(ncols=2, nrows=2)

N = 4

for i in range(N) :
  axi = fig.add_subplot(spec[i])
  annotate_axes(axi, f'ax {i}')



In [ ]:
i,j

In [ ]:
val_series


In [ ]:
#manual check time stamps
from datetime import datetime

def convert_timestamp(timestamp) :
  converted = None
  try : 
   # print("trying to convert")
    converted = datetime.fromtimestamp(int(timestamp))
  except Exception as e :
    print(e)
  
 # print(timestamp,"converted in : ",converted)

  return converted



df0["last_modified_t"].apply(lambda timestamp: convert_timestamp(timestamp))

## cutting off low freq fillrate

In [ ]:
cutoff = 10
df1_desc_select = df1_desc[df1_desc["fillRate%"]>cutoff]
df1_desc_select

In [ ]:
display(HTML(f"""<h1>Low freq cutoff : {df1_desc_select.shape[0]}""")) 

screenDF(df1_desc_select,25,15)



## droped to be investigated

In [ ]:
print(len(errors_dict),"errors to be investigated")

In [ ]:
to_review = list(set (df1.columns) - set(df1_desc_select.index.tolist()) )
df1[to_review].head()

fig,(ax0,ax1) = plt.subplots(1,2,figsize=(25,12))

import wordcloud

#not weighted

text = ' '.join(to_review)

wordcloud = WordCloud(background_color='white', width = 300, height=300, margin=2).generate(str(text))

ax0.imshow(wordcloud)
ax0.axis("off")

#weighted

df_lost = df1.drop(df1_desc_select.index.tolist(),axis=1)
lost_series = len(df_lost)-df_lost.isna().sum()
lost_dict = lost_series.to_dict()
lost_dict
wordcloud.background_color = 'black'
a = wordcloud.fit_words(lost_dict)
ax1.imshow(a)

ax1.axis("off")


plt.show()


## working on subset data : 

In [ ]:
df2 = df1[df1_desc_select.index.tolist()]

In [ ]:
df1['last_modified_t'].isna().sum()

In [ ]:
df1_desc_select.index.tolist()[:10]

In [ ]:
df1[df1_desc_select.index.tolist()[:10]]

In [ ]:
lostRate = list((map(lambda x,y : f"{100*(1-x/y):.0f}%" , df2.shape , df1.shape)))

display(HTML(f"<h2>lost data (r,c) : {lostRate}"))



In [ ]:
df2,desc2_dict,errors2_dict = clean1(df2)

In [ ]:
#
msno.dendrogram(df2)


In [ ]:
df1_desc_select == df2_desc

In [ ]:
df2_desc = pd.DataFrame(desc2_dict).transpose()
screenDF(df2_desc)
#df2_desc

In [ ]:
msno.heatmap(df2)
plt.show()